In [1]:
# # run this and then check the seed endings
# # only need to run this block one time to get the file and then may require some data cleaning

# #merges csv files
# import pandas as pd
# import glob
# import os

# # merging the files
# files_joined = os.path.join(r"C:\Users\allym\Documents\MQP\all_data", "data_*.csv")

# # Return a list of all joined files
# list_files = glob.glob(files_joined)
# print("** Merging multiple csv files into a single pandas dataframe **")
# # Merge files by joining all files
# df = pd.concat(map(pd.read_csv, list_files), ignore_index=True)
# data=pd.DataFrame(df)
# # print(data.columns.tolist())
# # print(df)
# data.to_csv(r"C:\Users\allym\Documents\MQP\all_data\all_data.csv")

In [2]:
## this script will be used to calculate the score using bayes average

####### to run this you will want to:
# 1. update the file to be your data file
# 2. update the trial increments
# 3. update the location of the file you are writing to
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None

#get correct path for files
__file__ = 'bayes_weighted_average'
absolutepath = os.path.abspath(__file__)
print(absolutepath) #should be MQP/app/python_scripts/bayes_weighted_average
fileDirectory = os.path.dirname(absolutepath)
print(fileDirectory) #should be MQP/app/python_scripts
#Path of parent directory
parentDirectory = os.path.dirname(fileDirectory)
print(parentDirectory) #should be MQP/app
#Navigate to all_data directory
all_data = os.path.join(parentDirectory, 'all_data') #the directory going to, change if something different
print(all_data) #should be MQP/app/all_data
pathOutput = os.path.join(all_data, 'all_data.csv') #the file reading from, change if something different
print(pathOutput) #should be MQP/app/all_data/all_data.csv

data = pd.read_csv(pathOutput)


/Users/victoriamirecki/Downloads/MQP/app/python_scripts/bayes_weighted_average
/Users/victoriamirecki/Downloads/MQP/app/python_scripts
/Users/victoriamirecki/Downloads/MQP/app
/Users/victoriamirecki/Downloads/MQP/app/all_data
/Users/victoriamirecki/Downloads/MQP/app/all_data/all_data.csv


FileNotFoundError: [Errno 2] No such file or directory: '/Users/victoriamirecki/Downloads/MQP/app/all_data/all_data.csv'

In [3]:
# update this set of trials list with the start and end point
# of the DIFFERENT trials that you run (we're using the indexing
# added on by running the above script to concatenate the files)
num1=0
num2=3499
sets_of_trials=[]
while (num2 <= 248499):
    [start_trial,end_trial]=[num1,num2]
    sets_of_trials.append([start_trial,end_trial])
    num1=num1+3500
    num2=num2+3500
# print(sets_of_trials)

this_ped_occupied = 0
this_area = 0
this_evaluation_metric = 0
total_ped_occupied = 0
total_area = 0
total_evaluation_metric = 0
avg_ped_occupied = 0
avg_area = 0
avg_evaluation_metric = 0
list_of_scores = []
data['bayes_weighted_avg_score'] = ''
ij_index = 0
m = 0

# calculating the averages to be used later on
total_area = sum(data['width_i']*data['width_ii'])
avg_area = total_area/len(data['width_i'])
# print(avg_area)

total_ped_occupied = sum(data['max[Child]']+(2*data['max[Adult]'])+(4*data['max[AdultBackpack]'])+(14*data['max[AdultBike]']))
avg_ped_occupied = total_ped_occupied/len(data['max[Child]'])
# print('avg_ped_occupied',avg_ped_occupied)

total_evaluation_metric = sum(data['evaluation_metric'])
avg_evaluation_metric = total_evaluation_metric/len(data['evaluation_metric'])
# print('avg_evaluation_metric',avg_evaluation_metric)

# for each set of trials from the list, get the index of the
# first trial to calculate this_ped_occupied and this_area 
index = data.index
for [i,j] in sets_of_trials:
    seed = data['seed']
    x = i
    eval_sum = 0
    conditioni = seed == i
    seedi_index = index[conditioni].tolist()
    
# this_ped_occupied is the number of cells occupied by pedestrians 
# so multiply num peds by number of cells that type of ped takes
# up for the given set of trials
    this_ped_occupied = data['max[Child]'][seedi_index]+(2*data['max[Adult]'][seedi_index])+(4*data['max[AdultBackpack]'][seedi_index])+(14*data['max[AdultBike]'][seedi_index])
#     print('this_ped_occupied',this_ped_occupied)

# this_area is the area of the grid for the given set of trials
    this_area = data['width_i'][seedi_index]*data['width_ii'][seedi_index]
#     print('this_area',this_area)

# this_evaluation_metric is the average of the evaluation metric
# for the given set of trials
    while (x <= j):
        seed = data['seed']
        conditionx = seed == x
        seedx_index = index[conditionx].tolist()
        eval_sum = int(eval_sum) + int(data['evaluation_metric'][seedx_index])
        x+=1
        
#     print(eval_sum)
    this_evaluation_metric = eval_sum/(j-i+1)
    
# calculating the weight for bayes weighted average (based on ped occupied and area)
    weight_ped_occupied = this_ped_occupied/(this_ped_occupied+avg_ped_occupied)
    weight_area = this_area/(this_area+avg_area)
    weight = weight_ped_occupied*weight_area

# this is the final score calculated using bayes weighted average
# lower score is better
    score = (weight*this_evaluation_metric)+((1-weight)*avg_evaluation_metric)
#     print(float(score))
    list_of_scores.append(float(score))
    
# writing the score to be the last column of the data csv
for [i,j] in sets_of_trials:
    seed = data['seed']
    conditioni = seed == i
    seedi_index = index[conditioni]
    conditionj = seed == j
    seedj_index = index[conditionj]
    while ((m<=seedj_index) and (m>=seedi_index)):
        data['bayes_weighted_avg_score'][m]=list_of_scores[ij_index]

        m = m + 1
    ij_index = ij_index + 1
data.to_csv(pathOutput, index=False)

i made it
